In [1]:
import os
import requests
import gzip
import shutil
import pandas as pd

# define dataset URLs you want to load
dataset_urls = {
    'title.basics': 'https://datasets.imdbws.com/title.basics.tsv.gz',
    'title.ratings': 'https://datasets.imdbws.com/title.ratings.tsv.gz',
    'name.basics':   'https://datasets.imdbws.com/name.basics.tsv.gz',
    'title.principals': 'https://datasets.imdbws.com/title.principals.tsv.gz',
    'title.crew': 'https://datasets.imdbws.com/title.crew.tsv.gz',
    'title.episode': 'https://datasets.imdbws.com/title.episode.tsv.gz',
    # add more datasets as needed
}

data_dir = 'imdb_data'
os.makedirs(data_dir, exist_ok=True)

for name, url in dataset_urls.items():
    gz_path = os.path.join(data_dir, f'{name}.tsv.gz')
    tsv_path = os.path.join(data_dir, f'{name}.tsv')

    # download
    print(f'Downloading {url} …')
    resp = requests.get(url, stream=True)
    with open(gz_path, 'wb') as f:
        shutil.copyfileobj(resp.raw, f)

    # decompress
    print(f'Decompressing {gz_path} → {tsv_path} …')
    with gzip.open(gz_path, 'rb') as f_in:
        with open(tsv_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    # optionally remove .gz file
    os.remove(gz_path)

    # load into pandas
    print(f'Loading {tsv_path} into DataFrame …')
    df = pd.read_csv(tsv_path, sep='\t', dtype=str, low_memory=False)
    print(name, 'loaded:', df.shape)
    # e.g. store to a dict:
    globals()[f'df_{name.replace(".", "_")}'] = df



Decompressing imdb_data\title.basics.tsv.gz → imdb_data\title.basics.tsv …
Loading imdb_data\title.basics.tsv into DataFrame …
title.basics loaded: (12097750, 9)
Decompressing imdb_data\title.ratings.tsv.gz → imdb_data\title.ratings.tsv …
Loading imdb_data\title.ratings.tsv into DataFrame …
title.ratings loaded: (1605930, 3)
Decompressing imdb_data\name.basics.tsv.gz → imdb_data\name.basics.tsv …
Loading imdb_data\name.basics.tsv into DataFrame …
name.basics loaded: (14905821, 6)
Decompressing imdb_data\title.principals.tsv.gz → imdb_data\title.principals.tsv …
Loading imdb_data\title.principals.tsv into DataFrame …
title.principals loaded: (96150630, 6)
Decompressing imdb_data\title.crew.tsv.gz → imdb_data\title.crew.tsv …
Loading imdb_data\title.crew.tsv into DataFrame …
title.crew loaded: (12097750, 3)
Decompressing imdb_data\title.episode.tsv.gz → imdb_data\title.episode.tsv …
Loading imdb_data\title.episode.tsv into DataFrame …
title.episode loaded: (9325795, 4)


##### the title.akas file was too big so we have to add it manually.

In [2]:
import pandas as pd

file_path = r"C:\Users\Eric\Desktop\ING 5\big data processing\project\imdb_data\title.akas.tsv"  # put your file path here

chunksize = 500_000
chunks = []

for chunk in pd.read_csv(file_path, sep="\t", dtype=str, chunksize=chunksize):
    chunks.append(chunk)

df_title_akas = pd.concat(chunks, ignore_index=True)



### Question

In [3]:
#How many total people in data set?
df_name_basics.shape[0]


14905821

In [4]:
#print line of person born that year
earliest_year = df_name_basics['birthYear'].replace('\\N', pd.NA).dropna().astype(int).min()
df_earliest = df_name_basics[df_name_basics['birthYear'] == str(earliest_year)]
df_earliest

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
737971,nm0784172,Lucio Anneo Seneca,4,65,writer,"tt0043802,tt0218822,tt0049203,tt0972562"


In [5]:
#How many years ago was this person born?
from datetime import datetime
current_year = datetime.now().year
year_ago = current_year - earliest_year
print("it was" ,year_ago, "years ago")

it was 2021 years ago


In [6]:
#Using only the data in the data set, determine if this date of birth correct.
df_earliest['deathYear'] = df_earliest['deathYear'].replace('\\N', pd.NA)
df_earliest['is_alive'] = df_earliest['deathYear'].isna()
df_earliest[['primaryName', 'birthYear', 'deathYear', 'is_alive']]

C:\Users\Eric\AppData\Local\Temp\ipykernel_11916\2810622323.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_earliest['deathYear'] = df_earliest['deathYear'].replace('\\N', pd.NA)
C:\Users\Eric\AppData\Local\Temp\ipykernel_11916\2810622323.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_earliest['is_alive'] = df_earliest['deathYear'].isna()


,primaryName,birthYear,deathYear,is_alive
737971,Lucio Anneo Seneca,4,65,False


With these verification, we can determine that this dateof birth is correct

In [7]:
#What is the most recent date of birth?
latest_year = df_name_basics['birthYear'].replace('\\N', pd.NA).dropna().astype(int).max()
df_latest = df_name_basics[df_name_basics['birthYear'] == str(latest_year)]
df_latest

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
7063566,nm16784939,Kyrah Ivy Jackson,2025,\N,actress,\N
11506326,nm5642311,Chase Ramsey,2025,\N,"actor,director,writer","tt17505010,tt14715170,tt4236770,tt17062324"


In [8]:
#What percentage of the people do not have a listed date of birth?
missing_births = df_name_basics['birthYear'].value_counts().get('\\N', 0)
total_people = df_name_basics.shape[0]
percentage_missing = (missing_births / total_people) * 100
print(f'Percentage of people without listed date of birth: {percentage_missing:.2f}%')

Percentage of people without listed date of birth: 95.57%


In [9]:
#What is the length of the longest "short" after 1900?

start_year = pd.to_numeric(df_title_basics['startYear'], errors='coerce')

df_after_1900 = df_title_basics[
    (start_year > 1900) &
    (df_title_basics['titleType'] == 'short')
]

runtime = pd.to_numeric(df_after_1900['runtimeMinutes'], errors='coerce')

longest_short = runtime.max()
print(f'Length of the longest "short" after 1900: {longest_short} minutes')

#print the line of the longest "short" after 1900
df_longest_short = df_after_1900[runtime == longest_short]
df_longest_short

Length of the longest "short" after 1900: 1311.0 minutes


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8520581,tt35509411,short,Our First Day,Our First Day,0,2025,\N,1311,"Drama,Short"


In [10]:
#What is the length of the shortest "movie" after 1900?
start_year = pd.to_numeric(df_title_basics['startYear'], errors='coerce')
df_movies_after_1900 = df_title_basics[
    (start_year > 1900) &
    (df_title_basics['titleType'] == 'movie')
]
runtime_movies = pd.to_numeric(df_movies_after_1900['runtimeMinutes'], errors='coerce')
shortest_movie = runtime_movies.min()
print(f'Length of the shortest "movie" after 1900: {shortest_movie} minutes')

df_longest_movie = df_movies_after_1900[runtime_movies == shortest_movie]
df_longest_movie

Length of the shortest "movie" after 1900: 1.0 minutes


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
24728,tt0025166,movie,George White's Scandals,George White's Scandals,0,1934,\N,1,"Comedy,Musical,Romance"
450320,tt0469119,movie,Love Trap,Love Trap,0,2005,\N,1,Drama
784824,tt0810779,movie,Bound by Blood,Bound by Blood,0,2007,\N,1,Action
821097,tt0848384,movie,Nikkatsu on Parade,Nikkatsu on Parade,0,1930,\N,1,Documentary
2548473,tt12893768,movie,If I Die Tomorrow,If I Die Tomorrow,0,2020,\N,1,Documentary
6249536,tt26348770,movie,Dancing Boy,Dancing Boy,0,2023,\N,1,Documentary
7722613,tt32276067,movie,Honest Vikky (Life Coach),Honest Vikky (Life Coach),1,2024,\N,1,Adult
9390623,tt39051124,movie,The Challenger: Comic brought to Life,The Challenger: Comic brought to Life,0,2025,\N,1,Animation


In [11]:
#List of all of the genres represented.

all_genres = set()
for genres in df_title_basics['genres'].dropna():
    for genre in genres.split(','):
        all_genres.add(genre)

all_genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western',
 '\\N'}

In [12]:
#What is the highest rated comedy "movie" in the dataset? Note, if there is a tie, the tie shall be broken by the movie with the most votes .

df_comedy_movies = df_title_basics[
    (df_title_basics['titleType'] == 'movie') &
    (df_title_basics['genres'].str.contains('Comedy', na=False))
]

df_comedy_with_ratings = pd.merge(
    df_comedy_movies,
    df_title_ratings,
    on='tconst',
    how='inner'
)
df_comedy_with_ratings['averageRating'] = pd.to_numeric(df_comedy_with_ratings['averageRating'], errors='coerce')
df_comedy_with_ratings['numVotes'] = pd.to_numeric(df_comedy_with_ratings['numVotes'], errors='coerce')
highest_rated_comedy = df_comedy_with_ratings.sort_values(
    by=['averageRating', 'numVotes'],
    ascending=[False, False]
)
display(highest_rated_comedy)

print("the highest rated comedy movie is", highest_rated_comedy.iloc[0]['primaryTitle'], "with a rating of", highest_rated_comedy.iloc[0]['averageRating'], "and", highest_rated_comedy.iloc[0]['numVotes'], "votes.")

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
49239,tt1423343,movie,Bob vs. Society,Bob vs. Society,0,2009,\N,90,Comedy,10.0,12
73430,tt5533084,movie,Happy Lucky Golden Tofu Panda Dragon Good Time...,Happy Lucky Golden Tofu Panda Dragon Good Time...,0,2017,\N,75,"Comedy,Documentary,Music",10.0,11
78937,tt8458418,movie,O La La,O La La,0,2018,\N,125,Comedy,10.0,6
51340,tt1543724,movie,Reck and Ima,Reck and Ima,0,2013,\N,91,Comedy,10.0,5
69498,tt38876234,movie,Kamen batyrov,Kamen batyrov,0,2025,\N,80,"Comedy,Fantasy,Mystery",10.0,5
...,...,...,...,...,...,...,...,...,...,...,...
27215,tt0199358,movie,"Belli, carucci e pettinati","Belli, carucci e pettinati",0,2000,\N,\N,Comedy,1.0,13
59243,tt23012498,movie,Our President's Money,Our President's Money,0,2022,\N,\N,Comedy,1.0,12
62484,tt27819521,movie,Camp Break 2,Farar Az Kamp 2,0,2012,\N,\N,Comedy,1.0,11
63222,tt28689690,movie,The Rat Catcher,Moosh Gir,0,2017,\N,87,Comedy,1.0,9


the highest rated comedy movie is Bob vs. Society with a rating of 10.0 and 12 votes.


In [13]:
#Who was the director of the movie Bob vs. Society?

df_bob_vs_society = df_title_basics[
    df_title_basics['primaryTitle'] == 'Bob vs. Society'
]

df_bob_vs_society_crew = pd.merge(
    df_bob_vs_society,
    df_title_crew,
    on='tconst',
    how='inner'
)
directors_ids = df_bob_vs_society_crew.iloc[0]['directors'].split(',')
directors = df_name_basics[df_name_basics['nconst'].isin(directors_ids)]
directors[['primaryName', 'nconst']]



,primaryName,nconst
8572341,David Pring-Mill,nm2301250


In [15]:
#List, if any, the alternate titles for the movie Bob vs. Society.

df_bob_vs_society_akas = df_title_akas[
    df_title_akas['titleId'] == df_bob_vs_society.iloc[0]['tconst']
]

df_bob_vs_society_akas.head()




,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
15198422,tt1423343,1,Bob vs. Society,\N,\N,original,\N,1
15198423,tt1423343,2,Bob vs. Society,GB,\N,imdbDisplay,\N,0
15198424,tt1423343,3,Bob vs. Society,US,\N,imdbDisplay,\N,0
